In [1]:
# CREATE CASSANDRA CONNECTION
import cassandra
from cassandra.cluster import Cluster
try: 
    cluster = Cluster(['127.0.0.1'])
    session = cluster.connect()
except Exception as e:
    print(e)

In [2]:
# TEST THE CONNECTION SHOULD ERROR
try:
    session.execute("""select * from db001""")
except Exception as e:
    print(e)

Error from server: code=2200 [Invalid query] message="No keyspace has been specified. USE a keyspace, or explicitly specify keyspace.tablename"


In [3]:
# CREATE A KEYSPACE TO DO OUR WORK
try:
    session.execute("""CREATE KEYSPACE IF NOT EXISTS udacity WITH REPLICATION = {'class': 'SimpleStrategy', 'replication_factor':1}""")
except Exception as e:
    print(e)

In [4]:
# SET THE KEYSPACE/SCHEMA NAME
try:
    session.set_keyspace('udacity')
except Exception as e:
    print(e)

In [5]:
# CREATE A TABLE(S) IN THE DATABASE
query = "CREATE TABLE IF NOT EXISTS music_library"
query = query + "(year int, artist_name text, album_name text, PRIMARY KEY (year, artist_name))"
try:
    session.execute(query)
except Exception as e:
    print(e)

query = "CREATE TABLE IF NOT EXISTS album_library"
query = query + "(artist_name text, year int, album_name text, PRIMARY KEY (artist_name, year))"
try:
    session.execute(query)
except Exception as e:
    print(e)

In [6]:
# INSERT SOME DATA INTO TABLE
query = "INSERT INTO music_library (year, artist_name, album_name)"
query = query + " VALUES (%s, %s, %s)"

query1 = "INSERT INTO album_library (artist_name, year, album_name)"
query1 = query1 + " VALUES (%s, %s, %s)"

try:
 session.execute(query, (1970, "The Beatles", "Let it Be"))
except Exception as e:
 print(e)

try:
 session.execute(query, (1965, "The Beatles", "Rubber Soul")),
except Exception as e:
 print(e)
try:
 session.execute(query, (1965, "The Who", "My Generation")),
except Exception as e:
 print(e)

try:
 session.execute(query, (1966, "The Monkees", "The Monkees")),
except Exception as e:
 print(e)

try:
 session.execute(query, (1970, "The Carpenters", "Close To ")),
except Exception as e:
 print(e)

try:
 session.execute(query1, ("The Beatles", 1970, "Let it Be")),
except Exception as e:
 print(e)

try:
 session.execute(query1, ("The Beatles", 1965, "Rubber Soul ")),
except Exception as e:
 print(e)

try:
 session.execute(query1, ("The Who", 1965, "My Generation")),
except Exception as e:
 print(e)

try:
 session.execute(query1, ("The Monkees", 1966, "The Monkees ")),
except Exception as e:
 print(e)

try:
 session.execute(query1, ("The Carpenters", 1970, "Close To You")),
except Exception as e:
 print(e)

Error from server: code=1100 [Coordinator node timed out waiting for replica nodes' responses] message="Operation timed out - received only 0 responses." info={'consistency': 'LOCAL_ONE', 'required_responses': 1, 'received_responses': 0, 'write_type': 'SIMPLE'}
Error from server: code=1100 [Coordinator node timed out waiting for replica nodes' responses] message="Operation timed out - received only 0 responses." info={'consistency': 'LOCAL_ONE', 'required_responses': 1, 'received_responses': 0, 'write_type': 'SIMPLE'}


### Inserting duplicate data into two tables might feel weird. 
- If I just normalized these tables, I wouldn't have to have extra copies! While this is true, remember there are no JOINS in Apache Cassandra. 
- For the benefit of high availibity and scalabity denormalization must be how this is done.

In [7]:
# VALIDATE MUSIC_LIBRARY DATA MODEL
query = "select * from music_library WHERE YEAR=1970"
try:
 rows = session.execute(query)
except Exception as e:
 print(e)

for row in rows:
 print (row.year, row.artist_name, row.album_name,)

1970 The Beatles Let it Be
1970 The Carpenters Close To 


In [8]:
# VALIDATE ALBUM_LIBRARY DATA MODEL
query = "select * from album_library WHERE ARTIST_NAME='The Beatles'"
try:
 rows = session.execute(query)
except Exception as e:
 print(e)

for row in rows:
 print (row.artist_name, row.year, row.album_name)

The Beatles 1965 Rubber Soul 
The Beatles 1970 Let it Be


In [9]:
# DROP THE CREATED TABLE FOR DEMO PURPOSES
query = "DROP table music_library"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

query = "DROP table album_library"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

errors={'127.0.0.1:9042': 'Client request timeout. See Session.execute[_async](timeout)'}, last_host=127.0.0.1:9042


In [10]:
# CLOSE SESSION AND CLUSTER CONNECTION
session.shutdown()
cluster.shutdown()